# Selección de características 

En este ejercicio seleccionaré las diferentes variables de una base de datos de vinos para poder determinar la calidad de algún vino, para esto hare uso de librerías especializadas. Se realizará un proceso de selección de características hacia adelante y de eliminación hacia atrás en una regresión lineal múltiple.

Los datos originales se obtuvieron de UCI Machine Learning Repository:   
https://archive.ics.uci.edu/dataset/186/wine+quality

Y se reportaron originalmente en "Modeling wine preferences by data mining from physicochemical properties" en la revista "Decision Support Systems":  
https://www.sciencedirect.com/science/article/abs/pii/S0167923609001377?via%3Dihub

Las ligas se encuentran si quieres verificar las bases de datos, si deseas probar la funcionalidad del código usa la base de datos que se proporciona como descargable ya que los datos fueron modificados para trabajar con ellos de manera más sencilla.

La base de datos cuenta con la siguiente información:  
  
* “acidezFija”. La acidez fija del vino, medida en gramos de ácido tartárico por decímetro
cúbico.  
* “acidezVolatil”. La acidez volátil del vino, medida en gramos de ácido acético por
decímetro cúbico.  
* “acidoCitrico”. Gramos de tácito cítrico por decímetro cúbico.  
* “azucarResidual”. Gramos de azúcar por decímetro cúbico.  
* “cloruros”. Gramos de cloruro de sodio por decímetro cúbico.  
* “dioxidoAzufreLibre”. Miligramos de dióxido de azufre libre por decímetro cúbico.  
* “dioxidoAzufreTotal”. Miligramos de dióxido de azufre total por decímetro cúbico.  
* “densidad”. Medida en gramos por centímetro cúbico.  
* “pH”. Valor del vino en la escala de pH.  
* “sulfatos”. Gramos de sulfato de potasio por decímetro cúbico.  
* “alcohol”. volumen percentil de alcohol en el vino.  
* “calidad”. Mediana de la calidad otorgada por al menos tres catadores, en escala del 0
(muy malo) al 10 (excelente)  


Iniciaremos importando el archivo CSV, posteriormente visualizaremos sus dimensiones y las primeras 5 filas de datos

In [25]:
import pandas as pd

df = pd.read_csv("Datos1_4.csv")

print("Dimensiones:", df.shape)
print("\nPrimeras 5 filas:\n",df.head(5))

Dimensiones: (1599, 12)

Primeras 5 filas:
    acidezFija  acidezVolatil  acidoCitrico  azucarResidual  cloruros  \
0         7.4           0.70          0.00             1.9     0.076   
1         7.8           0.88          0.00             2.6     0.098   
2         7.8           0.76          0.04             2.3     0.092   
3        11.2           0.28          0.56             1.9     0.075   
4         7.4           0.70          0.00             1.9     0.076   

   dioxidoAzufreLibre  dioxidoAzufreTotal  densidad    pH  sulfatos  alcohol  \
0                11.0                34.0    0.9978  3.51      0.56      9.4   
1                25.0                67.0    0.9968  3.20      0.68      9.8   
2                15.0                54.0    0.9970  3.26      0.65      9.8   
3                17.0                60.0    0.9980  3.16      0.58      9.8   
4                11.0                34.0    0.9978  3.51      0.56      9.4   

   calidad  
0        5  
1        5  
2  

Ahora separaremos los datos en datos de entrenamiento 80% y el 20% para datos de prueba y revisaremos que la cantidad de datos sea acorde a la cantidad de datos con los que contamos.

In [26]:
#Importa una funcion de una libreria
from sklearn.model_selection import train_test_split

#Divide los datos en datos de prueba y de entrenamiento
train, test = train_test_split(df, train_size = 0.8)

#Revisamos que tengamos la totalidad de los datos
print ("Suma de los datos:", train.shape[0] + test.shape[0])

Suma de los datos: 1599


Se separarán los datos en "x" y "y", y crearemos el modelo de sequential future selector de una librería especializada
como estimador usaremos la regresión lineal, donde se escogerán de 2 a 8 variables, hacia adelante, con una métrica de R^2 y una validación cruzada de 10.

In [27]:
#Importamos librerias y funciones
import numpy as np
from sklearn.linear_model import LinearRegression
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

#Separamos los datos en x y y de la serie de datos de entrenamiento
x_train = train.drop("calidad", axis= 1)
y_train = train["calidad"]

#Creamos el modelo de regresion lineal
lr = LinearRegression()

#Vaciamos las caracteristicas del modelo de sequential future selector
sfs = SFS(lr, k_features= (2,8), forward=True, scoring= 'r2', cv = 10 )

#Ejecutamos el selector 
sfs.fit(x_train,y_train)

#Obtenemos los nombres e indices de las variables selecionadas
featuresIndx = list(sfs.k_feature_idx_)
features = list(sfs.k_feature_names_)
print("Features(Variables selecionadas): \n", features)

Features(Variables selecionadas): 
 ['acidezVolatil', 'azucarResidual', 'cloruros', 'dioxidoAzufreLibre', 'dioxidoAzufreTotal', 'pH', 'sulfatos', 'alcohol']


Se entrenará un modelo el cual contenga únicamente las variables seleccionadas.

Se va a predecir la respuesta en las observaciones de prueba y evaluaremos la predicción del modelo usando R^2

In [28]:
from sklearn.metrics import r2_score


y_test = test["calidad"]
x_test = test.drop("calidad", axis = 1)


x_train_s1 = x_train[features]
x_test_s1 = x_test[features]

lr.fit(x_train_s1,y_train)


yhat = lr.predict(x_test_s1)

r2 = r2_score(y_test , yhat)

print("R^2 =", r2)

R^2 = 0.3039534622575165


Una selección de características hacia atrás será llevada a cabo con las variables que habíamos obtenido previamente, dando como resultado una selección de características múltiple.

In [29]:
#Creamos el modelo de regresion lineal
lr1 = LinearRegression()

#Vaciamos las caracteristicas del modelo de sequential future selector
sfs1 = SFS(lr, k_features= (2,5), forward=False, scoring= 'r2', cv = 10 )

#Ejecutamos el selector 
sfs1.fit(x_train_s1,y_train)

#Obtenemos los nombres e indices de las variables selecionadas
featuresIndx1 = list(sfs1.k_feature_idx_)
features1 = list(sfs1.k_feature_names_)
print("Features(Variables selecionadas): \n", features1)

Features(Variables selecionadas): 
 ['acidezVolatil', 'cloruros', 'pH', 'sulfatos', 'alcohol']


Con ese  modelo se obtendrá el nuevo valor de R^2

In [30]:
x_train_s2 = x_train[features1]
x_test_s2 = x_test[features1]

lr1.fit(x_train_s2,y_train)

yhat1 = lr1.predict(x_test_s2)

r2_mixto = r2_score(y_test,yhat1)

print("R^2 de la selección mixta de características =", r2_mixto)


R^2 de la selección mixta de características = 0.2853533967394576


Podemos observar que la R^2 obtenida con el modelo que contiene menos variables de entrenamiento describe de peor manera los datos de prueba con el modelo generado a partir de los datos de entrenamiento, este comportamiento es esperable. En general cuando las variables estén relacionadas si aumenta el número de estas los modelos describen mejor el comportamiento de los datos cuando se comparan en términos de R^2, no obstante podríamos caer en una sobre-optimización además hay que tomar en cuenta que más variables aumentan por mucho los requerimientos de computación necesaria, por lo cual, para evaluar de manera correcta el desempeño de ambos sistemas sería conveniente utilizar la R^2 ajustada la cual toma en cuenta la cantidad de variables del modelo.

¿Cuál modelo usar?
Es importante no caer en la fuga de datos, en este caso utilizar indirectamente tus datos de prueba para el entrenamiento del modelo, por lo cual sería conveniente haber separado los datos en un segmento extra y poder elegir cual modelo usar usando la R^2 ajustada para posteriormente obtener la R^2 del modelo seleccionado. 

Dicho esto, diría que el modelo con menor cantidad de variables representa mejor el comportamiento que buscamos analizar, pero es solo una conjetura, para estar seguros deberíamos de utilizar R^2 ajustada para estar seguros de nuestra afirmación, sería bueno analizar si el ahorro en computo es relevante.


Agradezco al lector por su atención:

-Armando Rdz.